In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, concatenate

In [6]:
model_input = Input(shape=(28,28,3))
pre = Dense(192)(model_input)

conv1 = Conv2D(64, 1, padding='same')(pre)

conv1_2 = Conv2D(96, 1, padding='same')(pre)  # 1x1
conv2 = Conv2D(128, 3, padding='same')(conv1_2)

conv1_3 = Conv2D(16, 1, padding='same')(pre) # 1x1
conv3 = Conv2D(32, 5, padding='same')(conv1_3)

pool = MaxPooling2D(pool_size=(3,3), strides=1, padding='same')(pre)
conv1_4 = Conv2D(32, 1, padding='same')(pool) # 1x1

model_output = concatenate([conv1, conv2, conv3, conv1_4])
model = Model(inputs=model_input, outputs=model_output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 28, 28, 3)]          0         []                            
                                                                                                  
 dense_2 (Dense)             (None, 28, 28, 192)          768       ['input_3[0][0]']             
                                                                                                  
 conv2d_6 (Conv2D)           (None, 28, 28, 96)           18528     ['dense_2[0][0]']             
                                                                                                  
 conv2d_8 (Conv2D)           (None, 28, 28, 16)           3088      ['dense_2[0][0]']             
                                                                                              

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Activation, add

In [8]:
model_input = Input(shape=(28,28,3))

x = Conv2D(128, 3, padding='same', activation='relu')(model_input)

conv = Conv2D(64, 3, padding='same', activation='relu')(x)
conv = Conv2D(128, 3, padding='same')(conv)

y = add([conv, x])
y = Activation('relu')(y)

model = Model(inputs=model_input, outputs=y)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 28, 28, 3)]          0         []                            
                                                                                                  
 conv2d_11 (Conv2D)          (None, 28, 28, 128)          3584      ['input_4[0][0]']             
                                                                                                  
 conv2d_12 (Conv2D)          (None, 28, 28, 64)           73792     ['conv2d_11[0][0]']           
                                                                                                  
 conv2d_13 (Conv2D)          (None, 28, 28, 128)          73856     ['conv2d_12[0][0]']           
                                                                                            

In [9]:
# CIFAR10 이미지 분류 모델을 Residual Block을 이용해서 구현

from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

inputs = Input(shape=(32,32,3))

conv = Conv2D(32, 3, activation='relu', padding='same')(inputs)
# x = conv(inputs)
x = BatchNormalization()(conv)

# Residual Block을 2개
# 1번째 잔차블록
conv = Conv2D(32, 3, activation='relu', padding='same')(x)
y = BatchNormalization()(conv)

conv = Conv2D(32, 3, padding='same')(y)
y = BatchNormalization()(conv)

x = add([x, y])
x = Activation('relu')(x)

# 2번째 잔차블록
conv = Conv2D(32, 3, activation='relu', padding='same')(x)
y = BatchNormalization()(conv)

conv = Conv2D(32, 3, padding='same')(y)
y = BatchNormalization()(conv)

x = add([x, y])
x = Activation('relu')(x)

x = MaxPooling2D()(x)
y = Flatten()(x)

y = Dense(512, activation='relu')(y)
outputs = Dense(10, activation='softmax')(y)

model = Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d_14 (Conv2D)          (None, 32, 32, 32)           896       ['input_5[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 32, 32, 32)           128       ['conv2d_14[0][0]']           
 Normalization)                                                                                   
                                                                                                  
 conv2d_15 (Conv2D)          (None, 32, 32, 32)           9248      ['batch_normalization[0]

In [10]:
from tensorflow.keras.datasets import cifar10
(train_X, train_y), (test_X, test_y) = cifar10.load_data()
train_X = train_X / 255.0
test_X = test_X / 255.0
print(train_X.shape, test_X.shape)

170498071/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3) (10000, 32, 32, 3)


In [12]:
from tensorflow.keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir='logs', histogram_freq=1, embeddings_freq=1)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.fit(train_X, train_y, validation_data=(test_X, test_y),
          epochs=30, batch_size=200,
          callbacks=[tensorboard])

Epoch 1/30
250/250 [==============================] - 17s 44ms/step - loss: 2.8891 - accuracy: 0.3536 - val_loss: 1.9299 - val_accuracy: 0.2880
Epoch 2/30
250/250 [==============================] - 10s 39ms/step - loss: 1.3028 - accuracy: 0.5343 - val_loss: 1.4620 - val_accuracy: 0.4821
Epoch 3/30
250/250 [==============================] - 10s 40ms/step - loss: 1.0821 - accuracy: 0.6150 - val_loss: 1.1814 - val_accuracy: 0.5855
Epoch 4/30
250/250 [==============================] - 10s 40ms/step - loss: 0.9200 - accuracy: 0.6764 - val_loss: 1.0354 - val_accuracy: 0.6384
Epoch 5/30
250/250 [==============================] - 10s 39ms/step - loss: 0.8261 - accuracy: 0.7116 - val_loss: 1.0023 - val_accuracy: 0.6515
Epoch 6/30
250/250 [==============================] - 11s 42ms/step - loss: 0.7498 - accuracy: 0.7370 - val_loss: 0.9242 - val_accuracy: 0.6827
Epoch 7/30
250/250 [==============================] - 10s 41ms/step - loss: 0.6880 - accuracy: 0.7589 - val_loss: 0.9407 - val_accuracy:

In [1]:
from tensorflow.keras.applications import ResNet50
model = ResNet50()

102967424/102967424 [==============================] - 1s 0us/step


In [3]:
from tensorflow.keras.preprocessing import image
# 이미지 라이브러리: OpenCV, PIL
img = image.load_img("YellowLabradorLooking_new.jpg", target_size=(224,224))
x = image.img_to_array(img).reshape(-1, 224, 224, 3)
pred = model.predict(x)
print(pred)

1/1 [==============================] - 2s 2s/step
[[4.33807872e-06 1.52540497e-06 6.96436155e-06 1.65774927e-05
  1.08696186e-05 1.06768039e-06 6.00921965e-07 1.21360699e-05
  4.55568579e-06 3.19667379e-06 6.29502281e-08 6.10987342e-07
  6.12321912e-07 3.46508045e-07 1.45254512e-06 1.84455487e-07
  8.59957197e-07 2.49889399e-06 5.77762137e-07 3.60813061e-07
  1.13777332e-07 2.26864313e-06 8.09308574e-07 2.68382701e-05
  6.21508389e-06 1.01057401e-06 4.53236879e-07 7.03097271e-07
  8.61081503e-07 7.07881918e-06 1.55072894e-06 3.08694689e-06
  9.61662636e-07 9.06477271e-06 5.07999084e-06 2.02075299e-07
  1.43051659e-06 5.39846087e-06 1.11721545e-06 2.96722442e-06
  1.11545239e-06 1.05331310e-06 1.69657540e-06 1.68659119e-06
  1.36466610e-06 2.53124790e-06 3.83963925e-06 3.62667879e-06
  1.89725233e-05 5.31404794e-07 2.28100043e-06 2.60282127e-06
  8.52118774e-06 6.79506627e-07 7.75838942e-07 5.58854481e-06
  7.05373338e-07 4.55737364e-07 1.12284783e-06 2.12057762e-06
  1.00536681e-06 8.0

In [4]:
from tensorflow.keras.applications.resnet50 import decode_predictions
decode_predictions(pred, top=5)

35363/35363 [==============================] - 0s 0us/step


[[('n02099712', 'Labrador_retriever', 0.26897743),
  ('n02108089', 'boxer', 0.15149544),
  ('n02099849', 'Chesapeake_Bay_retriever', 0.10756537),
  ('n02089867', 'Walker_hound', 0.067649655),
  ('n02104029', 'kuvasz', 0.046868663)]]

In [5]:
from tensorflow.keras.applications import ResNet50
cnn_model = ResNet50(input_shape=(224,224,3), include_top=False)
cnn_model.trainable = True

94765736/94765736 [==============================] - 1s 0us/step


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

In [9]:
model = Sequential()
model.add(cnn_model)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(100, activation='softmax')) # 분류 클래스 라벨 개수는 100개
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dense_1 (Dense)             (None, 1024)              102761472 
                                                                 
 dense_2 (Dense)             (None, 100)               102500    
                                                                 
Total params: 126451684 (482.37 MB)
Trainable params: 126398564 (482.17 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________
